### Dépendances

In [2]:
import pandas as pd
import wikipedia
from wikipedia import WikipediaPage
wikipedia.set_lang("fr")

### Traitement 

Nous n'avions pu récupérer initialement que peu d'auteurs par scraping (peu de pages permettant de récupérer des listes d'auteurs). Nous avons donc été chercher, sur chaque page de mouvements littéraires scrapés, des auteurs qui y appartiennent, par lecture. Ceci nous permet de passer à 622 auteurs et nous amène cependant à abandonner certains mouvements littéraires pour lesquels il était difficile d'établir une liste d'auteurs à partir de leur page wikipédia.

In [3]:
df=pd.read_csv('ress/liste_auteurs_manuelle.csv',index_col=[0]) #index_col pour éviter la colonne 'Unnamed: 0' de pandas
df.head()

,mouvement,auteurs
0,hagiographie,"['https://fr.wikipedia.org/wiki/Jean_Moschus',..."
1,troubadour,['https://fr.wikipedia.org/wiki/Arnaut-Guilhem...
2,fabliau,"['https://fr.wikipedia.org/wiki/Jean_Bodel', '..."
3,roman courtois,['https://fr.wikipedia.org/wiki/Chr%C3%A9tien_...
4,rhetoriqueurs,"['/wiki/Guillaume_de_Machaut', '/wiki/Famille_..."


In [4]:
# Caractères spéciaux
rep=[['À','%C3%80'],	
['Á','%C3%81'],
['Â','%C3%82'],
['Ã','%C3%83'],
['Ä','%C3%84'],
['Å','%C3%85'],
['Æ','%C3%86'],
['Ç','%C3%87'],
['È','%C3%88'],
['É','%C3%89'],
['Ê','%C3%8A'],
['Ë','%C3%8B'],
['Ì','%C3%8C'],
['Í','%C3%8D'],
['Ï','%C3%8F'],
['Ñ','%C3%91'],
['Ò','%C3%92'],
['Ó','%C3%93'],
['Ô','%C3%94'],
['Õ','%C3%95'],
['Ö','%C3%96'],
['Ù','%C3%99'],
['Ú','%C3%9A'],
['Û','%C3%9B'],
['Ü','%C3%9C'],
['Ý','%C3%9D'],
['à','%C3%A0'],
['á','%C3%A1'],
['â','%C3%A2'],
['ã','%C3%A3'],
['ä','%C3%A4'],
['å','%C3%A5'],
['æ','%C3%A6'],
['ç','%C3%A7'],
['è','%C3%A8'],
['é','%C3%A9'],
['ê','%C3%AA'],
['ë','%C3%AB'],
['ì','%C3%AC'],
['í','%C3%AD'],
['î','%C3%AE'],
['ï','%C3%AF'],
['ñ','%C3%B1'],
['ò','%C3%B2'],
['ó','%C3%B3'],
['ô','%C3%B4'],
['õ','%C3%B5'],
['ö','%C3%B6'],	
['ù','C3%B9'],
['ú','%C3%BA'],	
['û','%C3%BB'],
['ü','%C3%BC'],
['ý','%C3%BD'],
['ÿ','%C3%BF']]

On applique <code>clean2</code> qui remet les guillemets après le traitement de <code>get_aut</code> afin d'éviter de confondre les différents types de guillemets :<br>
- chaque élément de la colonne "auteurs" est un string donc on enlève les guillemets et [, ou ].
- aut_list est bien une liste de strings; on remplace le codage en pourcentage par son équivalent <'>.

In [5]:
def aut_clean(text):
    for car in rep:
        text=text.replace(car[1],car[0])
    return text
    
def get_aut(L):
    Lclean=[]
    for item in L.split(','):
        item=aut_clean(item)
        item=item.replace('[','')
        item=item.replace(']','')
        item=item.replace("'",'')
        item=item.replace(' ','')
        if 'wikipedia' in item:
            item=item.replace('https://fr.wikipedia.org/wiki/','')
            Lclean.append(item)
        else:
            item=item.replace('/wiki/','')
            Lclean.append(item)
    return Lclean

df['aut_list']=df['auteurs'].apply(lambda x : get_aut(x))

In [6]:
def clean2(L):
    Lv=[]
    for text in L:
        text=text.replace('%27',"'")
        Lv.append(text)
    return Lv

df['aut_list']=df['aut_list'].apply(lambda x : clean2(x))
df.head()

,mouvement,auteurs,aut_list
0,hagiographie,"['https://fr.wikipedia.org/wiki/Jean_Moschus',...","[Jean_Moschus, Jacques_de_Voragine, Paul_Guéri..."
1,troubadour,['https://fr.wikipedia.org/wiki/Arnaut-Guilhem...,"[Arnaut-Guilhem_de_Marsan, Raimon_d'Alayrac, N..."
2,fabliau,"['https://fr.wikipedia.org/wiki/Jean_Bodel', '...","[Jean_Bodel, Hugues_III_d'Oisy, Gautier_de_Coi..."
3,roman courtois,['https://fr.wikipedia.org/wiki/Chr%C3%A9tien_...,[Chrétien_de_Troyes]
4,rhetoriqueurs,"['/wiki/Guillaume_de_Machaut', '/wiki/Famille_...","[Guillaume_de_Machaut, Famille_Robertet#Jean_(..."


### Création du dataframe 

In [7]:
mvmdata,autdata=[],[]

for mvm in df['mouvement']:
    num_ligne=df[df['mouvement']==mvm].index[0]

    #liste des mouvements
    for i in range(len(df['aut_list'][num_ligne])):
        mvmdata.append(mvm)
    
    #liste des auteurs
    for aut in df['aut_list'][num_ligne]:
        autdata.append(aut)    

dat={'Nom':autdata,'Mouvement':mvmdata}
tab=pd.DataFrame(dat)
print(tab)

                          Nom     Mouvement
0                Jean_Moschus  hagiographie
1         Jacques_de_Voragine  hagiographie
2     Paul_Guérin_(religieux)  hagiographie
3    Société_des_Bollandistes  hagiographie
4    Arnaut-Guilhem_de_Marsan    troubadour
..                        ...           ...
357     Jacques_Rabemananjara     Négritude
358        Léon-Gontran_Damas     Négritude
359              Guy_Tirolien     Négritude
360               Birago_Diop     Négritude
361             René_Depestre     Négritude

[362 rows x 2 columns]


Il y a un élément problématique : '/w/index.php?title=Félix-Quentin_Caffiau&action=edit&redlink=1'(trouvé à l'aide de deux fonctions définies plus bas).<br>
On le supprime.

In [8]:
index_el=tab[tab['Nom']=='/w/index.php?title=Félix-Quentin_Caffiau&action=edit&redlink=1'].index[0]
tab.drop(index_el,axis=0,inplace=True)
tab

,Nom,Mouvement
0,Jean_Moschus,hagiographie
1,Jacques_de_Voragine,hagiographie
2,Paul_Guérin_(religieux),hagiographie
3,Société_des_Bollandistes,hagiographie
4,Arnaut-Guilhem_de_Marsan,troubadour
...,...,...
357,Jacques_Rabemananjara,Négritude
358,Léon-Gontran_Damas,Négritude
359,Guy_Tirolien,Négritude
360,Birago_Diop,Négritude


In [9]:
# tab.to_csv('auteurs_man.csv')

#### Partie test pour trouver l'élément problématique

In [10]:
# Tester si la liste des auteurs fonctionne bien ; si on enlève la condition sur aut alors on a une erreur. 
# On utilise la fonction <sum> pour trouver quels élément de tab['Nom'] créent une erreur.

# ! Très long !
testaut=[WikipediaPage(aut).summary for aut in tab['Nom']]

In [60]:
# Fonction utilisée pour trouver l'élément problématique
def sum(L):
    erreurs=[]
    for aut in L:
        try:
            summar=WikipediaPage(aut)
        except:
            erreurs.append(aut)
    return erreurs